In [1]:
from cim.loaders import Parameter, ConnectionParameters
from cim.loaders.blazegraph import BlazegraphConnection
from cim.loaders.gridappsd import GridappsdConnection, get_topology_response
from cim.models.distributed_model import DistributedModel
import cim.data_profile as cim
import json


In [2]:
# Blazegraph connection for running inside the container
params = ConnectionParameters([Parameter(key="url", value="http://blazegraph:8080/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params)

In [3]:
# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params)

In [4]:
#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
# feeder_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
#feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
#feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node

In [5]:
feeder = cim.Feeder(mRID=feeder_mrid)

In [6]:
# topology_response = get_topology_response(feeder_mrid)
with open('topo_message.json') as topo_file:
    topology_response = json.load(topo_file)


In [7]:
network = DistributedModel(connection=bg, feeder=feeder, topology=topology_response)

LoadBreakSwitch
Fuse
LoadBreakSwitch
Breaker
Recloser
ACLineSegment
TransformerTank
ACLineSegment
PowerTransformer
PowerTransformer
TransformerTank
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
TransformerTank
ACLineSegment
ConnectivityNode
ConnectivityNode
ConnectivityNode
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
TransformerTank
TransformerTank
ACLineSegment
ConnectivityNode
Breaker


EnergyConsumer
EnergyConsumer
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
TransformerTank
PowerTransformer
ConnectivityNode
Fuse
EnergyConsumer
PowerElectronicsConnection
EnergyConsumer
EnergyConsumer
PowerElectronicsConnection
ConnectivityNode
PowerTransformer
LinearShuntCompensator
ACLineSegment
ConnectivityNode
ConnectivityNode
LoadBreakSwitch
LinearShuntCompensator
ACLineSegment
ConnectivityNode
ConnectivityNode
LoadBreakSwitch
TransformerTank
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
LoadBreakSwitch
LoadBreakSwitch
Recloser
EnergyConsumer
PowerElectronicsConnection
PowerElectronicsConnection
ConnectivityNode
TransformerTank


In [8]:
network.switch_areas[2].get_all_attributes(cim.LinearShuntCompensator)
network.switch_areas[0].get_all_attributes(cim.ACLineSegment)
network.get_all_attributes(cim.PowerTransformer)


In [9]:
network.switch_areas[0].secondary_areas[0].get_all_attributes(cim.EnergyConsumer)
network.switch_areas[0].secondary_areas[0].get_all_attributes(cim.EnergyConsumerPhase)

In [14]:
network.switch_areas[0].secondary_areas[0].typed_catalog[cim.EnergyConsumerPhase]

{'_82D805E6-CDB2-4E37-8D33-C5977DB1C353': EnergyConsumerPhase(mRID='_82D805E6-CDB2-4E37-8D33-C5977DB1C353', aliasName=None, description=None, name='645_B', Names=[], AssetDatasheet=None, Assets=[], Controls=[], Location=Location(mRID='_7DB84AEE-89EE-4C77-BEFA-E58CB10FADA1', aliasName=None, description=None, name=None, Names=[], Assets=[], CoordinateSystem=None, Fault=[], Measurements=[], PowerSystemResources=[]), Measurements=[], p='170000', phase='http://iec.ch/TC57/CIM100#SinglePhaseKind.B', q='125000', EnergyConsumer=EnergyConsumer(mRID='_47E52220-4596-4AF0-9724-0167B72A4DB8', aliasName=None, description=None, name=None, Names=[], AssetDatasheet=None, Assets=[], Controls=[], Location=None, Measurements=[], inService=None, Faults=[], OperationalLimitSet=[], BaseVoltage=None, SvStatus=[], Terminals=[], customerCount=None, grounded=None, p=None, phaseConnection=None, q=None, EnergyConsumerPhase=[], House=None, LoadResponse=None, PowerCutZone=None)),
 '_C163738E-2A54-4886-9741-76C6E6FAB

In [10]:
#ridAPPSD Agent Side
network.typed_catalog[cim.PowerTransformer]

{'_1E6B5C97-C4E8-4CED-B9A5-6E69F389DA93': PowerTransformer(mRID='_1E6B5C97-C4E8-4CED-B9A5-6E69F389DA93', aliasName=None, description=None, name='xfm1', Names=[], AssetDatasheet=None, Assets=[], Controls=[], Location=Location(mRID='_6D30F773-D772-465C-9A4C-41624B283C2C', aliasName=None, description=None, name=None, Names=[], Assets=[], CoordinateSystem=None, Fault=[], Measurements=[], PowerSystemResources=[]), Measurements=[], inService=None, Faults=[], OperationalLimitSet=[], BaseVoltage=None, SvStatus=[], Terminals=[], vectorGroup='Yy', PowerTransformerEnd=[PowerTransformerEnd(mRID='_B0352ED5-349E-4511-9663-EC1131757376', aliasName=None, description=None, name=None, Names=[], endNumber=None, grounded=None, rground=None, xground=None, BaseVoltage=None, CoreAdmittance=None, FromMeshImpedance=[], PhaseTapChanger=None, RatioTapChanger=None, Terminal=None, ToMeshImpedance=[], connectionKind=None, phaseAngleClock=None, r=None, ratedS=None, ratedU=None, PowerTransformer=None), PowerTransform

In [11]:
json.loads(network.switch_areas[0].__dumps__(cim.ACLineSegment))


{'_0BBD0EA3-F665-465B-86FD-FC8B8466AD53': {'mRID': '_0BBD0EA3-F665-465B-86FD-FC8B8466AD53',
  'aliasName': '',
  'description': '',
  'name': '645646',
  'Names': [],
  'AssetDatasheet': '',
  'Assets': [],
  'Controls': [],
  'Location': {'mRID': '_89C21411-D107-454A-93B0-0905893DD27D',
   'aliasName': None,
   'description': None,
   'name': None,
   'Names': [],
   'Assets': [],
   'CoordinateSystem': None,
   'Fault': [],
   'Measurements': [],
   'PowerSystemResources': []},
  'Measurements': ['_2d9ed148-d492-4e7d-b9af-81a407e74a59',
   '_4e30ae83-418c-4cfb-99eb-5572e00b4ec4',
   '_5f1dfa9f-e754-471d-9c74-c1a9441e95a3',
   '_84f75532-a514-4355-bf0e-2f8d82c736ae'],
  'inService': '',
  'Faults': [],
  'OperationalLimitSet': [],
  'BaseVoltage': {'mRID': '4160',
   'aliasName': None,
   'description': None,
   'name': None,
   'Names': [],
   'nominalVoltage': None,
   'ConductingEquipment': [],
   'TopologicalNode': [],
   'TransformerEnds': []},
  'SvStatus': [],
  'Terminals': ['

In [12]:
# App Side
# contextmanager.typed_catalog[cim.ACLineSegment]

In [15]:
json.loads(network.switch_areas[0].secondary_areas[0].__dumps__(cim.EnergyConsumerPhase))

{'_82D805E6-CDB2-4E37-8D33-C5977DB1C353': {'mRID': '_82D805E6-CDB2-4E37-8D33-C5977DB1C353',
  'aliasName': '',
  'description': '',
  'name': '645_B',
  'Names': [],
  'AssetDatasheet': '',
  'Assets': [],
  'Controls': [],
  'Location': {'mRID': '_7DB84AEE-89EE-4C77-BEFA-E58CB10FADA1',
   'aliasName': None,
   'description': None,
   'name': None,
   'Names': [],
   'Assets': [],
   'CoordinateSystem': None,
   'Fault': [],
   'Measurements': [],
   'PowerSystemResources': []},
  'Measurements': [],
  'p': '170000',
  'phase': 'http://iec.ch/TC57/CIM100#SinglePhaseKind.B',
  'q': '125000',
  'EnergyConsumer': {'mRID': '_47E52220-4596-4AF0-9724-0167B72A4DB8',
   'aliasName': None,
   'description': None,
   'name': None,
   'Names': [],
   'AssetDatasheet': None,
   'Assets': [],
   'Controls': [],
   'Location': None,
   'Measurements': [],
   'inService': None,
   'Faults': [],
   'OperationalLimitSet': [],
   'BaseVoltage': None,
   'SvStatus': [],
   'Terminals': [],
   'customerCo

In [ ]:
cim.EnergyConsumerPhase().__dict__